In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch.nn.functional as F

In [2]:
# model
model = "allenai/OLMo-1B-hf"
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForCausalLM.from_pretrained(model,
                                             output_hidden_states=False,
                                             device_map="auto")

# end of sentence/text token padding
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.71G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [3]:
model.config

OlmoConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "allenai/OLMo-1B-hf",
  "architectures": [
    "OlmoForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "clip_qkv": null,
  "eos_token_id": 50279,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 2048,
  "model_type": "olmo",
  "num_attention_heads": 16,
  "num_hidden_layers": 16,
  "num_key_value_heads": 16,
  "pad_token_id": 1,
  "pretraining_tp": 1,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": true,
  "torch_dtype": "float32",
  "transformers_version": "4.48.3",
  "use_cache": true,
  "vocab_size": 50304
}

In [4]:
# list of cities extracted from https://en.wikipedia.org/wiki/The_Eras_Tour using ChatGPT with GPT 4o
cities = ['Glendale', 'Paradise', 'Arlington', 'Tampa', 'Houston', 'Atlanta', 'Nashville', 'Philadelphia', 'Foxborough',
          'East Rutherford', 'Chicago', 'Detroit', 'Pittsburgh', 'Minneapolis', 'Cincinnati', 'Kansas City', 'Denver',
          'Seattle', 'Santa Clara', 'Inglewood', 'Mexico City', 'Buenos Aires', 'Rio de Janeiro', 'São Paulo', 'Tokyo',
          'Melbourne', 'Sydney', 'Singapore', 'Nanterre', 'Stockholm', 'Lisbon', 'Madrid', 'Décines-Charpieu',
          'Edinburgh', 'Liverpool', 'Cardiff', 'London', 'Dublin', 'Amsterdam', 'Zürich', 'Milan', 'Gelsenkirchen',
          'Hamburg', 'Munich', 'Warsaw', 'Vienna', 'London', 'Miami Gardens', 'New Orleans', 'Indianapolis', 'Toronto',
          'Vancouver']

In [5]:
cities.index("Rio de Janeiro")

22

In [6]:
def next_word_prediction_probs(prompt, n=10):
  fw = False
  inp_tok = tokenizer(prompt,
                        padding=True,
                        return_tensors="pt").to(next(model.parameters()).device)
  input_ids = inp_tok["input_ids"]
  logits = model(**inp_tok).logits[:, -1, :]
  probs = F.log_softmax(logits, dim=-1).detach()
  vals = [[tokenizer.decode(tk.item()),
             logits[0][tk.item()].tolist()] for tk in torch.argsort(probs, descending=True)[:, :n][0]]
  return vals

In [7]:
# this small function will clean the list of tokens (sorted by probabilities)
# to remove whitespace, non-alphabetical characters, and very short tokens.
def clean(probs):
    probs = [w[0].lstrip() for w in probs]
    probs = [w for w in probs if w.isalpha()]
    probs = [w for w in probs if len(w) > 2]
    return probs

In [8]:
print(clean(next_word_prediction_probs(f'My favorite recording artist is',n=100))[13])

Taylor


In [9]:
# I Was Dreaming
print("I Was Dreaming\n\nOf",end=" ")
for i, city in enumerate(cities):
  token = clean(next_word_prediction_probs(f"I couldn't imagine Taylor Swift being anything but a",n=100))[i]
  print(f'a {token} star',end='/ ')

I Was Dreaming

Of a good star/ a great star/ a pop star/ a super star/ a country star/ a nice star/ a huge star/ a bitch star/ a div star/ a feminist star/ a sweet star/ a beautiful star/ a fan star/ a terrible star/ a bad star/ a sweetheart star/ a girl star/ a woman star/ a total star/ a little star/ a big star/ a wonderful star/ a genius star/ a very star/ a self star/ a pretty star/ a lovely star/ a perfect star/ a complete star/ a mega star/ a horrible star/ a slut star/ a Disney star/ a bit star/ a lady star/ a narciss star/ a perfection star/ a kind star/ a fantastic star/ a class star/ a star star/ a talented star/ a princess star/ a true star/ a fabulous star/ a dou star/ a heart star/ a Christian star/ a straight star/ a queen star/ a lesbian star/ a hypocr star/ 

In [10]:
# 52 Cities Where I Can't
print("52 Cities Where I Can't\n")
for i, city in enumerate(cities):
    print(f"I put on my",end=" ")
    print(clean(next_word_prediction_probs(f'In {city} All my girls are wearing their lace and their',n=100))[i],end=", ")
    print("feeling",end = " ")
    print(clean(next_word_prediction_probs(f'After the Taylor Swift show in {city}, I was filled with feelings of',n=100))[i],end=",\n")
    print("afraid of",end=" ")
    print(clean(next_word_prediction_probs(f'I almost died at the Taylor Swift concert in {city} by',n=100))[i],end=" ")
    print("and want to",end=" ")
    print(clean(next_word_prediction_probs(f'When I watch Taylor Swift dancing on stage in {city} I want to',n=100))[i],end=" ")
    print("but I can't.\n")

52 Cities Where I Can't

I put on my dresses, feeling excitement,
afraid of falling and want to dance but I can't.

I put on my white, feeling gratitude,
afraid of myself and want to scream but I can't.

I put on my white, feeling excitement,
afraid of getting and want to scream but I can't.

I put on my dresses, feeling joy,
afraid of having and want to get but I can't.

I put on my high, feeling love,
afraid of being and want to get but I can't.

I put on my sexy, feeling disappointment,
afraid of not and want to know but I can't.

I put on my skirts, feeling disappointment,
afraid of eating and want to feel but I can't.

I put on my pretty, feeling hope,
afraid of trying and want to say but I can't.

I put on my little, feeling pride,
afraid of trying and want to throw but I can't.

I put on my little, feeling awe,
afraid of trying and want to cry but I can't.

I put on my pear, feeling inadequ,
afraid of jumping and want to say but I can't.

I put on my sexy, feeling being,
afraid 